# Домашнее задание к вебинару 7. Сверточные нейронные сети для анализа текста

Задание:

Взять отзывы за лето
1. Учим conv сеть для классификации
2. Рассмотреть 2-а варианта нейронной сети: 
2.1 Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/
2.2 Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)
Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сетью, что получилось лучше

In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")
from nltk.probability import FreqDist

import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, MaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.losses import SparseCategoricalCrossentropy
from keras.callbacks import EarlyStopping  
from tensorflow.keras.utils import to_categorical

from sklearn.preprocessing import LabelEncoder
import gensim

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


**Загрузка и предобработка данных**  
Заранее перевела данные в формат csv

In [4]:
df = pd.read_csv('/content/отзывы за лето.csv')
df.head(5)

,Unnamed: 0,Rating,Content,Date
0,0,5,It just works!,2017-08-14
1,1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,2,5,Отлично все,2017-08-14
3,3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,4,5,"Очень удобно, работает быстро.",2017-08-14


In [5]:
df.drop('Unnamed: 0', axis = 1, inplace = True)

In [6]:
df.head(2)

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14


In [7]:
df['Rating'].value_counts()

5    14586
1     2276
4     2138
3      911
2      748
Name: Rating, dtype: int64

Заметен значительный дисбаланс классов

In [8]:
df_train, df_test = train_test_split(df, test_size=0.33, random_state=42)
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_train.shape, df_test.shape

((13841, 3), (6818, 3))

In [9]:
df_train.head(2)

,Rating,Content,Date
0,5,Наконец-то исправили эту чушь с неоргинальной ...,2017-08-09
1,5,Удобно в использовании,2017-07-27


In [10]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['Content'] = df_train['Content'].apply(preprocess_text)
df_test['Content'] = df_test['Content'].apply(preprocess_text)

In [11]:
train_corpus = " ".join(df_train["Content"])
train_corpus = train_corpus.lower()

In [12]:
tokens = word_tokenize(train_corpus)

In [13]:
tokens[:20]

['наконецтый',
 'исправить',
 'чушь',
 'снеоргинальный',
 'прошивка',
 'приложение',
 'удобно',
 'пользоваться',
 'удобно',
 'использование',
 'отлично',
 'класс',
 'удобно',
 'замечательный',
 'приложение',
 'довольный',
 'влиять',
 'работа',
 'устройство',
 'приходиться']

Отфильтруем данные и соберём в корпус N наиболее частых токенов

In [14]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [15]:
max_words = 800 #поставила число больше, чем в файле занятия

dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [106]:
len(tokens_filtered_top)

799

In [16]:
tokens_filtered_top[:10]

['приложение',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'хороший',
 'отличный',
 'телефон',
 'супер']

In [17]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [18]:
vocabulary['супер']

10

In [19]:
max_len = 100 # значение выше, чем на занятии

def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [20]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["Content"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["Content"]], dtype=np.int32)

In [21]:
# Теперь скорректируем y_train и y_test, чтобы их можно было подавать в сеть
num_classes = df['Rating'].nunique()
num_classes

5

In [22]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(df_train['Rating']) 
test_enc_labels = le.transform(df_test['Rating'])
le.classes_

array([1, 2, 3, 4, 5])

In [46]:
y_train = to_categorical(train_enc_labels, num_classes)
y_test = to_categorical(test_enc_labels, num_classes)

**Нейронная сеть с предобученными векторами в слое Embedding**

In [24]:
# !wget http://vectors.nlpl.eu/repository/20/180.zip

--2022-06-16 15:47:07--  http://vectors.nlpl.eu/repository/20/180.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.181
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 484452317 (462M) [application/zip]
Saving to: ‘180.zip.2’

180.zip.2           100%[===================>] 462.01M  27.0MB/s    in 18s     

2022-06-16 15:47:26 (25.6 MB/s) - ‘180.zip.2’ saved [484452317/484452317]



In [25]:
# !unzip 180.zip

Archive:  180.zip
replace meta.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: meta.json               
replace model.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: model.bin               
replace model.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: model.txt               
replace README? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: README                  


In [60]:
#загрузим предобученные векторы
word_vectors = gensim.models.KeyedVectors.load_word2vec_format('/content/model.bin', binary=True) .vectors 

In [61]:
len(word_vectors), len(word_vectors[1]), word_vectors[1]

(249333,
 300,
 array([ 4.5154796e+00,  3.8297811e+00, -6.9469607e-01, -1.3958697e+00,
        -6.8803182e+00,  2.5444067e+00, -1.3319516e+00, -1.0333906e+00,
        -5.6126447e+00,  7.4659262e+00,  1.4538134e-02, -3.5751505e+00,
         3.5980268e+00, -3.0855367e+00, -4.0427369e-01,  3.5216520e+00,
        -5.8490925e+00, -3.0425388e-01,  1.6812118e+00,  2.9846013e+00,
         1.7240332e+00,  3.9353080e+00,  4.0222373e+00, -1.1100594e+00,
        -1.5550443e+00, -2.1131717e-01, -1.0955086e+00, -5.9671655e+00,
         1.1800621e-01, -6.5576923e-01,  4.9077687e+00,  2.3885634e+00,
         1.3892661e+00,  3.8770003e+00,  1.4573048e+00, -8.8545698e-01,
        -7.6731639e+00, -1.4003915e+00, -9.4185764e-01,  1.2374935e+00,
        -6.8622670e+00, -1.0298090e+00, -3.9164896e+00,  2.3562472e+00,
        -2.6401494e+00, -7.1391735e+00, -5.8870583e+00, -4.6986256e+00,
         2.3890433e+00,  6.7100887e+00, -2.9966933e-01,  4.6237164e+00,
         1.9630557e+00,  1.0540729e+00, -3.987162

In [63]:
max_words

800

In [75]:
#обрежем word_vectors по размеру эмбеддингов (взяла длину эмбединга = 256)  по количеству слов, которые у нас есть в словаре
word_vectors_emd = [word_vectors[i][:256] for i in range(max_words)]
word_vectors_emd[1]

array([ 4.5154796e+00,  3.8297811e+00, -6.9469607e-01, -1.3958697e+00,
       -6.8803182e+00,  2.5444067e+00, -1.3319516e+00, -1.0333906e+00,
       -5.6126447e+00,  7.4659262e+00,  1.4538134e-02, -3.5751505e+00,
        3.5980268e+00, -3.0855367e+00, -4.0427369e-01,  3.5216520e+00,
       -5.8490925e+00, -3.0425388e-01,  1.6812118e+00,  2.9846013e+00,
        1.7240332e+00,  3.9353080e+00,  4.0222373e+00, -1.1100594e+00,
       -1.5550443e+00, -2.1131717e-01, -1.0955086e+00, -5.9671655e+00,
        1.1800621e-01, -6.5576923e-01,  4.9077687e+00,  2.3885634e+00,
        1.3892661e+00,  3.8770003e+00,  1.4573048e+00, -8.8545698e-01,
       -7.6731639e+00, -1.4003915e+00, -9.4185764e-01,  1.2374935e+00,
       -6.8622670e+00, -1.0298090e+00, -3.9164896e+00,  2.3562472e+00,
       -2.6401494e+00, -7.1391735e+00, -5.8870583e+00, -4.6986256e+00,
        2.3890433e+00,  6.7100887e+00, -2.9966933e-01,  4.6237164e+00,
        1.9630557e+00,  1.0540729e+00, -3.9871624e+00,  2.4700487e+00,
      

In [101]:
#инициализируем веса в эмбеддинге
initializer = tf.keras.initializers.Constant(word_vectors_emd)

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=256, embeddings_initializer = initializer, input_length=max_len)) 
                    #inputdim -- размер словаря, outputdim -- длина вектора, input_length -- длина входной последовательности
                    #на вход: (батч, input_lenght), на выходе: (батч, input_lenght, output_dim)

model.add(Conv1D(64, 3, padding = 'same'))
                   #128 -- количество 1D-фильтров, длина фильтра -- 3
                   #на выходе (количество объектов, 128 - длина нового эмбединга)

model.add(MaxPool1D(2))  

model.add(Activation("relu"))


model.add(Conv1D(128, 3, padding = 'same'))

model.add(MaxPool1D(2))

model.add(Activation("relu"))
                    #применяем функцию активации к выходу предыдущего слоя

model.add(GlobalMaxPool1D())
                    # в каждой свертке для каждого элемента эмбединга оставляем максимум - получаем один 128-м эмбеддинг

model.add(Dense(128))
               #128-количество выходов
model.add(Activation("relu"))
model.add(Dense(64))
model.add(Activation("relu"))

model.add(Dense(num_classes))
                #num_classes = 5 -- количество выходов
model.add(Activation('softmax'))
                #преобразуем вектор в рапределение вероятностей

In [102]:
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_14 (Embedding)    (None, 100, 256)          204800    
                                                                 
 conv1d_24 (Conv1D)          (None, 100, 64)           49216     
                                                                 
 max_pooling1d_23 (MaxPoolin  (None, 50, 64)           0         
 g1D)                                                            
                                                                 
 activation_60 (Activation)  (None, 50, 64)            0         
                                                                 
 conv1d_25 (Conv1D)          (None, 50, 128)           24704     
                                                                 
 max_pooling1d_24 (MaxPoolin  (None, 25, 128)          0         
 g1D)                                                

In [103]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [104]:
# tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
# early_stopping=EarlyStopping(monitor='val_loss', min_delta = 0.001)  

# Убрала callbacks, чтобы посмотреть изменение метрик

epochs = 10
batch_size = 512

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    # callbacks=[early_stopping]
                    )

Epoch 1/10
25/25 [==============================] - 17s 660ms/step - loss: 0.9756 - accuracy: 0.6934 - val_loss: 0.8278 - val_accuracy: 0.7184
Epoch 2/10
25/25 [==============================] - 16s 649ms/step - loss: 0.7857 - accuracy: 0.7374 - val_loss: 0.7902 - val_accuracy: 0.7314
Epoch 3/10
25/25 [==============================] - 16s 650ms/step - loss: 0.7306 - accuracy: 0.7512 - val_loss: 0.7555 - val_accuracy: 0.7357
Epoch 4/10
25/25 [==============================] - 16s 650ms/step - loss: 0.6787 - accuracy: 0.7693 - val_loss: 0.7342 - val_accuracy: 0.7444
Epoch 5/10
25/25 [==============================] - 16s 651ms/step - loss: 0.6351 - accuracy: 0.7820 - val_loss: 0.7823 - val_accuracy: 0.7379
Epoch 6/10
25/25 [==============================] - 16s 655ms/step - loss: 0.5944 - accuracy: 0.8016 - val_loss: 0.7394 - val_accuracy: 0.7451
Epoch 7/10
25/25 [==============================] - 16s 657ms/step - loss: 0.5463 - accuracy: 0.8164 - val_loss: 0.7610 - val_accuracy: 0.7329

In [105]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test f1_score:', score[1])

14/14 [==============================] - 2s 146ms/step - loss: 0.8286 - accuracy: 0.7479


Test loss: 0.8286218047142029
Test f1_score: 0.7478733062744141


**Нейронная сеть с обучаемым слоем Embedding**

In [77]:
max_len

100

In [78]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=256, input_length=max_len)) 
                    #inputdim -- размер словаря, outputdim -- длина вектора, input_length -- длина входной последовательности
                    #на вход: (батч, input_lenght), на выходе: (батч, input_lenght, output_dim)

model.add(Conv1D(64, 3, padding = 'same'))
                   #128 -- количество 1D-фильтров, длина фильтра -- 3
                   #на выходе (количество объектов, 128 - длина нового эмбединга)

model.add(MaxPool1D(2))  

model.add(Activation("relu"))


model.add(Conv1D(128, 3, padding = 'same'))

model.add(MaxPool1D(2))

model.add(Activation("relu"))
                    #применяем функцию активации к выходу предыдущего слоя

model.add(GlobalMaxPool1D())
                    # в каждой свертке для каждого элемента эмбединга оставляем максимум - получаем один 128-м эмбеддинг

model.add(Dense(128))
               #128-количество выходов
model.add(Activation("relu"))
model.add(Dense(64))
model.add(Activation("relu"))

model.add(Dense(num_classes))
                #num_classes = 5 -- количество выходов
model.add(Activation('softmax'))
                #преобразуем вектор в рапределение вероятностей

In [79]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, 100, 256)          204800    
                                                                 
 conv1d_16 (Conv1D)          (None, 100, 64)           49216     
                                                                 
 max_pooling1d_15 (MaxPoolin  (None, 50, 64)           0         
 g1D)                                                            
                                                                 
 activation_40 (Activation)  (None, 50, 64)            0         
                                                                 
 conv1d_17 (Conv1D)          (None, 50, 128)           24704     
                                                                 
 max_pooling1d_16 (MaxPoolin  (None, 25, 128)          0         
 g1D)                                                

In [80]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [44]:
# tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
# early_stopping=EarlyStopping(monitor='val_loss', min_delta = 0.001)  

# Убрала callbacks, чтобы посмотреть изменение метрик

epochs = 10
batch_size = 512

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    # callbacks=[early_stopping]
                    )

Epoch 1/10
25/25 [==============================] - 19s 707ms/step - loss: 1.1364 - accuracy: 0.6798 - val_loss: 0.9619 - val_accuracy: 0.7011
Epoch 2/10
25/25 [==============================] - 17s 680ms/step - loss: 0.8389 - accuracy: 0.7093 - val_loss: 0.7347 - val_accuracy: 0.7473
Epoch 3/10
25/25 [==============================] - 16s 636ms/step - loss: 0.6720 - accuracy: 0.7680 - val_loss: 0.7112 - val_accuracy: 0.7487
Epoch 4/10
25/25 [==============================] - 16s 636ms/step - loss: 0.6321 - accuracy: 0.7799 - val_loss: 0.6992 - val_accuracy: 0.7523
Epoch 5/10
25/25 [==============================] - 17s 694ms/step - loss: 0.6121 - accuracy: 0.7849 - val_loss: 0.7049 - val_accuracy: 0.7516
Epoch 6/10
25/25 [==============================] - 16s 655ms/step - loss: 0.5970 - accuracy: 0.7915 - val_loss: 0.7163 - val_accuracy: 0.7502
Epoch 7/10
25/25 [==============================] - 16s 637ms/step - loss: 0.5819 - accuracy: 0.7962 - val_loss: 0.7164 - val_accuracy: 0.7502

In [47]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test f1_score:', score[1])

14/14 [==============================] - 3s 213ms/step - loss: 0.7032 - accuracy: 0.7752


Test loss: 0.7032281160354614
Test f1_score: 0.7751539945602417


**Вывод:** сеть с обучаемыми эмбеддингами (не инициализиованными) показала результат лучше, нежели сеть с инициализированными эмбеддингами